In [5]:
from typing import Optional, Dict
from smolagents import CodeAgent, tool, LiteLLMModel , GradioUI,OpenAIServerModel
import requests
import os
import time
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
@tool
def scrape_realtor(state: str, city_name: str, num_pages: Optional[int] = 2) -> Dict[str, any]:
    """Scrapes realtor.com for agent information in specified city and state
    
    Args:
        state: State abbreviation (e.g., 'CA', 'NY')
        city_name: City name with hyphens instead of spaces (e.g., 'buffalo')
        num_pages: Number of pages to scrape (default: 2)
    """
    try:
        # Initialize results
        results = []         # Names
        phone_results = []   # Phone numbers
        office_results = []  # Office names
        pages_scraped = 0
        
        # Set up headers
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Accept-Language": "en-US,en;q=0.5",
            "Connection": "keep-alive"
        }

        # Process pages
        for page in range(1, num_pages + 1):
            # Construct URL
            if page == 1:
                url = f'https://www.realtor.com/realestateagents/{city_name}_{state}/'
            else:
                url = f'https://www.realtor.com/realestateagents/{city_name}_{state}/pg-{page}'
            
            print(f"Scraping page {page}...")
            
            # Get page content
            r = requests.get(url, headers=headers)
            if r.status_code != 200:
                return {"error": f"Failed to access page {page}: Status code {r.status_code}"}

            soup = BeautifulSoup(r.text, features="html.parser")
            
            # Find all agent cards
            agent_cards = soup.find_all('div', class_='agent-list-card')
            
            for card in agent_cards:
                # Find name
                name_elem = card.find('div', class_='agent-name')
                if name_elem:
                    name = name_elem.text.strip()
                    if name and name not in results:
                        results.append(name)
                        print(f"Found agent: {name}")

                # Find phone
                phone_elem = card.find('a', {'data-testid': 'agent-phone'}) or \
                            card.find(class_='btn-contact-me-call') or \
                            card.find('a', href=lambda x: x and x.startswith('tel:'))
                
                if phone_elem:
                    phone = phone_elem.get('href', '').replace('tel:', '').strip()
                    if phone:
                        phone_results.append(phone)
                        print(f"Found phone: {phone}")

                # Get office/company name
                office_elem = card.find('div', class_='agent-group') or \
                            card.find('div', class_='text-semibold')
                if office_elem:
                    office = office_elem.text.strip()
                    office_results.append(office)
                    print(f"Found office: {office}")
                else:
                    office_results.append("")
            
            pages_scraped += 1
            time.sleep(2)  # Rate limiting

        if not results:
            return {"error": "No agents found. The website structure might have changed or no results for this location."}

        # Return structured data
        return {
            "names": results,
            "phones": phone_results,
            "offices": office_results,
            "total_agents": len(results),
            "pages_scraped": pages_scraped,
            "city": city_name,
            "state": state
        }
        
    except Exception as e:
        return {"error": f"Scraping error: {str(e)}"}

In [7]:
@tool
def save_to_csv(data: Dict[str, any], filename: Optional[str] = None) -> str:
    """Saves scraped realtor data to CSV file
    
    Args:
        data: Dictionary containing scraping results
        filename: Optional filename (default: cityname.csv)
    """
    try:
        if "error" in data:
            return f"Error: {data['error']}"
            
        if not filename:
            filename = f"{data['city'].replace('-', '')}.csv"
            
        # Ensure all lists are of equal length
        max_length = max(len(data['names']), len(data['phones']), len(data['offices']))
        
        # Pad shorter lists with empty strings
        data['names'].extend([""] * (max_length - len(data['names'])))
        data['phones'].extend([""] * (max_length - len(data['phones'])))
        data['offices'].extend([""] * (max_length - len(data['offices'])))
        
        # Create DataFrame with just names, phones, and offices
        df = pd.DataFrame({
            'Names': data['names'],
            'Phone': data['phones'],
            'Office': data['offices']
        })
        
        df.to_csv(filename, index=False, encoding='utf-8')
        return f"Data saved to {filename}. Total entries: {len(df)}"
        
    except Exception as e:
        return f"Error saving CSV: {str(e)}"


In [8]:
deepseek_model =  OpenAIServerModel(
            model_id="deepseek-r1:7b",
            api_base="http://localhost:11434/v1",
            api_key="ollama"
        )

    # Create agent with tools
agent = CodeAgent(
    tools=[scrape_realtor, save_to_csv],
    model=deepseek_model,
    additional_authorized_imports=["pandas", "bs4", "time"]
)

In [ ]:
result = agent.run("""
Thought: Let's scrape realtor data
Code:
```python
# Scrape realtor data
data = scrape_realtor(state="NY", city_name="buffalo", num_pages=2)

# Save to CSV
if "error" not in data:
    result = save_to_csv(data)
    print(result)
else:
    print(f"Error: {data['error']}")
```
""")
    
print(result)
GradioUI(agent).launch()

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Thought: Let's scrape realtor data                                                                              │
│ Code:                                                                                                           │
│ ```python                                                                                                       │
│ # Scrape realtor data                                                                                           │
│ data = scrape_realtor(state="NY", city_name="buffalo", num_pages=2)                                             │
│                                                                                                                 │
│ # Save to CSV                                                                                                   │
│ if "error" not in data:                                                                                         │
│     result = save_to_csv(data)                                                                                  │
│     print(result)                                                                                               │
│ else:                                                                                                           │
│     print(f"Error: {data['error']}")                                                                            │
│ ```                                                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - deepseek-r1:7b ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
<think>
Okay, so I've been given this Python code that's supposed to scrape realtor data. Let me try to figure out what it 
does and what's going on here.

First part of the code is a call to a function called scrape_realtor. It has three parameters: state set to "NY", 
city_name as "buffalo", and num_pages set to 2. Then, if there's no key 'error' in the data dictionary returned by 
this function, it proceeds to save the data to a CSV file using another function called save_to_csv. If an error is
present, it catches that and prints the error message.

Hmm, I'm guessing that the scrape_realtor function makes some HTTP requests and parses HTML content from realtor 
websites to extract data about properties in Buffalo, NY. Since it's limited to 2 pages (num_pages=2), maybe each 
page has multiple listings.

Wait, but how does the data structure look? If there's an 'error' key, perhaps that means the scraping failed for 
some reason—like a network error or a bad HTML response. So before saving, the code checks if the 'error' key 
exists to handle it gracefully.

The save_to_csv function then writes this scraped data into a CSV file. I'm not exactly sure what keys are in the 
data dictionary from scrape_realtor, but likely each dictionary represents a realtor listing with attributes like 
price, number of beds, baths, square footage, etc., and maybe also URL or other info.

I wonder if I should validate this code. Maybe think about possible issues—like why would there be an error? Is the
website down, or are there 404 pages for certain states or cities? Also, is num_pages correctly handling multiple 
page scraper?

Another thought: since the data might include URLs, and sometimes realtor sites require logging in to scrape data. 
If that's the case, then this code wouldn't work because it can't access those listings without user 
authentication.

Also, maybe there are pagination issues. Since only 2 pages, perhaps some properties are on page 1 or 2, but if it 
doesn't handle loading more pages due to JavaScript or APIs requiring interaction, it might not get all data.

I should also check how the save_to_csv function is implemented. If it's using pandas, that would help structure 
the CSV properly with headers and rows. But without seeing its code, I can only assume.

Potential steps to improve:
1. Error handling: Maybe add more detailed error messages or logging.
2. Check if all pages are being parsed correctly—maybe some pages return empty data beyond a certain point.
3. Add authentication if the site requires user input for scraping.
4. Test the code with different parameters and see how it handles various states, cities, and page numbers.

I think one thing might be bothering me is assuming that scrape_realtor's response doesn't have an 'error' key 
except when something goes wrong. But perhaps in some scenarios, like if a property listing has errors on the 
website (e.g., unsupported elements), it could return an error even without exceptions? Not sure.

Also, I should consider that sometimes scraping might change over time as HTML structures evolve. So if 
scrape_realtor is making assumptions about the page structure, future changes could break the scraper unless it's 
properly updated.

Another point: The response from the server might have different content types or might require certain headers to 
be sent by the scraper client—like adding user-agent information for realtor sites which may block scrapers 
otherwise. Not sure if this code includes that, as a basic scraper.

So in summary, my thought process is:
- Acknowledge that the code's intent is to scrape realtor data from Buffalo, NY.
- Question why there is an 'error' key—maybe it's for failed requests elsewhere besides what might be expected.
- Consider possible issues like authentication needed, incorrect pagin

[Step 0: Duration 562.34 seconds| Input tokens: 89 | Output tokens: 1,145]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━